In [1]:
# from selenium import webdriver 
# from selenium.webdriver.common.by import By
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import time as tm
import os
import matplotlib.pyplot as plt
import yfinance as yf
import numpy as np
from math import log 

# Nombres de las compañias

In [2]:
companys=pd.read_excel('compañias_bmv.xlsx')
symbols=list(companys['Symbol'])
names=list(companys['Company'])
names=[name[:-1] if  name.endswith('.') else name for name in names ]
# sorted(names)

In [7]:
companys

,Symbol,Company
0,DDD.MX,3D Systems Corporation
1,MMM.MX,3M Company
2,AGNC.MX,AGNC Investment Corp.
3,ALPEKA.MX,"ALPEK, S.A.B. de C.V."
4,APE.MX,AMC ENTERTAINMENT HOLDINGS INC
...,...,...
501,ZM.MX,"Zoom Video Communications, Inc."
502,BLUE.MX,"bluebird bio, Inc."
503,EBAY.MX,eBay Inc.
504,EXPI.MX,"eXp World Holdings, Inc."


# Trayendo la información de  yahoo finance

In [8]:
def get_info_compamys(*ncompanys,**kwargs):
    nsymbs=[  symbols[names.index(com)]  if com not in symbols  else com for com in ncompanys]
    today=tm.localtime()
    today='-'.join([str(i) for i in list(today)[:3]])
    last_year=str(int(today[:4])-1)+today[4:]
    data = yf.download(tickers=sorted(nsymbs), start=last_year, end=today)
    def rendimiento(comp):return [log(comp[i]/comp[i-1]) for i in range(1,len(comp))]
    cols=list(data.Close)
    dates=list(str(i).split()[0] for i in data.index)
    df_rend=pd.DataFrame([rendimiento(list(data.Close[c])) for c in cols]).T
    df_rend.columns=cols
    monto=kwargs['monto_invertir']
    return df_rend

In [36]:
Datos=get_info_compamys( 'Grupo Bimbo, S.A.B. de C.V',
                       'Grupo Televisa, S.A.B',
                        'WMT.MX',
                        'Kimberly-Clark de México, S. A. B. de C. V',
                        'Grupo Carso, S.A.B. de C.V', 'CEMEX, S.A.B. de C.V',monto_invertir=100)
Datos

[*********************100%***********************]  6 of 6 completed


(     BIMBOA.MX  CEMEXCPO.MX  GCARSOA1.MX  KIMBERA.MX  TLEVISACPO.MX    WMT.MX
 0    -0.003351    -0.017732    -0.043560   -0.031794      -0.032053 -0.006244
 1    -0.005544    -0.003089    -0.015729    0.005514       0.005834  0.003886
 2    -0.019243    -0.003099     0.014964   -0.010182      -0.029518  0.003367
 3     0.011270     0.003717     0.038007   -0.010878       0.008353  0.000000
 4    -0.010461    -0.024414    -0.000442    0.001182       0.003361 -0.012835
 ..         ...          ...          ...         ...            ...       ...
 248  -0.002039    -0.018062    -0.008308    0.006868      -0.001466  0.004284
 249   0.001495     0.016868     0.013416   -0.025753       0.014205  0.005823
 250  -0.002857    -0.041469    -0.006949   -0.017204      -0.012373 -0.020805
 251  -0.031415    -0.026500    -0.023966   -0.040624      -0.024839  0.003766
 252  -0.023614    -0.014166    -0.020558   -0.019682      -0.019330  0.008650
 
 [253 rows x 6 columns], 100)

In [64]:
def last_price(lista_tickers):
    ticker_yahoo = yf.download(lista_tickers,start='2022-08-30')
    data = ticker_yahoo.Close.iloc[-2]
    return list(data)
last_prices=last_price(symbols)
last_prices[:20]

[*********************100%***********************]  506 of 506 completed


[2961.510009765625,
 429.95001220703125,
 1010.0,
 265.0,
 3181.389892578125,
 4.449999809265137,
 2708.0400390625,
 60.0,
 2278.0,
 2050.0,
 137.14999389648438,
 181.50999450683594,
 1608.0,
 12.300000190734863,
 7526.0,
 3119.659912109375,
 1760.0,
 2090.0,
 199.99000549316406,
 497.30999755859375]

In [47]:
df_last_prices=pd.DataFrame([(com,round(lp,2)) for com, lp in zip(names,last_prices)])
df_last_prices.columns=['Compañia','Precio']
df_last_prices


,Compañia,Precio
0,3D Systems Corporation,2961.51
1,3M Company,429.95
2,AGNC Investment Corp,1010.00
3,"ALPEK, S.A.B. de C.V",265.00
4,AMC ENTERTAINMENT HOLDINGS INC,3181.39
...,...,...
501,"Zoom Video Communications, Inc",111.00
502,"bluebird bio, Inc",999.57
503,eBay Inc,2210.46
504,"eXp World Holdings, Inc",1530.00
